# This notebook will be mainly used for the capstone project

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-bb8cf13c-f77b-4338-bcc4-b914d53e9a21',
    'IBM_API_KEY_ID': 'NpKJ5Sbd8ZN_JG0z5n7bCEwjR_-31Uh9lrua5DcnOR65',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'courseraapplieddatasciencecapston-donotdelete-pr-pctpvbbkf8nyfu',
    'FILE': 'toronto_districts.csv'
}

In [4]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-bb8cf13c-f77b-4338-bcc4-b914d53e9a21',
    'IBM_API_KEY_ID': 'NpKJ5Sbd8ZN_JG0z5n7bCEwjR_-31Uh9lrua5DcnOR65',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'courseraapplieddatasciencecapston-donotdelete-pr-pctpvbbkf8nyfu',
    'FILE': 'toronto_geoJSON.geojson'
}


## Webscraping Approach 1


### Download all the dependencies that we will need.

In [5]:
# Import libraries
# library to handle data in a vectorized manner
import numpy as np

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import csv
from datetime import datetime


In [6]:
# library for data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes 

# convert an address into latitude and longitude values
import geopy
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment this line to install support for maps in your environment
!conda install -c conda-forge folium=0.5.0 --yes

# map rendering library
import folium 

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Note: I used a different, but equal, web site for web scrapping than suggested in the instructions

In [7]:
# Set the URL you want to webscrape from
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'

# Connect to the URL
response = requests.get(url)

# Parse HTML and save to BeautifulSoup object
soup = BeautifulSoup(response.text,"html.parser")

# pause the code for a sec
time.sleep(1)

In [8]:
# .findAll returns list of all found elements 

# search soup object for all 'a' tags on the web page that have a title
# full_tag = soup.findAll('a', title=True)

# search object for all 'a' tags on the web page that have href
full_tag = soup.findAll('a', title=True)

# full_tag

###  OPTIONAL: use this code to save list of found web strings to a csv file
example:
   
with open("c:/toronto_neigh.csv", "w") as fp:
   
     writer = csv.writer(fp)
   
     writer.writerows(full_tag)


In [9]:
# save full_tag list of strings to a Pandas dataframe
raw_neigh_df = pd.DataFrame(full_tag)

# add a column to the dataframe to use for validating neighborhood names; use to 'sort' the web listings in Excel
raw_neigh_df ['valid'] = 0


In [10]:
# show the dimensions of the web scrapping dataframe
raw_neigh_df.shape
raw_neigh_df.head(10)

,0,valid
0,"<a class=""internal"" href=""/wiki/File:Roncesval...",0
1,"<a href=""/wiki/Roncesvalles,_Toronto"" title=""R...",0
2,"<a href=""/wiki/Metropolitan_Toronto"" title=""Me...",0
3,"<a href=""/wiki/Amalgamation_of_Toronto"" title=...",0
4,"<a href=""/wiki/York,_Toronto"" title=""York, Tor...",0
5,"<a href=""/wiki/East_York"" title=""East York"">Ea...",0
6,"<a href=""/wiki/North_York"" title=""North York"">...",0
7,"<a href=""/wiki/Etobicoke"" title=""Etobicoke"">Et...",0
8,"<a href=""/wiki/Scarborough,_Toronto"" title=""Sc...",0
9,"<a href=""/wiki/Old_Toronto"" title=""Old Toronto...",0


In [11]:
# save df to a csv file
# example: df.to_csv('test.csv', index=False, header=False

# save the dataframe to a csv file to be used for editing with Excel functions
raw_neigh_df.to_csv('coursera_capstone_toronto_web_scrapping.csv', index=False, header=None)

## Create Neighborhoods Dataframe

#### Create a dataset that contains the 9 Districts and the neighborhoods that exist in each district

#### read coursera_capstone_neighborhood.csv file into a pandas dataframe


In [27]:
# after web scrapping, use Excel functions to edit csv file. Then, get the data into a dataframe
input_df = pd.read_csv('toronto_districts_neighborhoods_valid.csv', header=None)

In [28]:
# show the size of the input_df
input_df.shape

(197, 3)

In [29]:
input_df.head()

,0,1,2
0,Downtown Core (Central),Alexandra Park,1
1,Downtown Core (Central),Baldwin Village,1
2,Downtown Core (Central),Cabbagetown,1
3,Downtown Core (Central),Chinatown,1
4,Downtown Core (Central),Church and Wellesley,1


In [30]:
# change column names to be used in geojson file
input_df.columns = ['District', 'Neighborhood', 'Valid']

In [31]:
# drop column 'Valid' from the dataframe; it will not be used
input_df = input_df.drop('Valid', axis=1)

In [32]:
# show dataframe of districts and neighborhoods
input_df.head()

,District,Neighborhood
0,Downtown Core (Central),Alexandra Park
1,Downtown Core (Central),Baldwin Village
2,Downtown Core (Central),Cabbagetown
3,Downtown Core (Central),Chinatown
4,Downtown Core (Central),Church and Wellesley


## Check the Dataframe of Districts and Neighborhoods

In [33]:
# make sure that the input_df dataset has all 9 districts and 197 neighborhoods.
print('The dataframe has {} Districts and {} neighborhoods.'.format(
        len(input_df['District'].unique()),
        input_df.shape[0]
    )
)


The dataframe has 9 Districts and 197 neighborhoods.


### Convert pandas 'input_df' to a csv file; will be input file for geopy to find latitude and longitude data

In [ ]:
# import pandas as pd
# create toronto_data csv file using input_df dataframe. 
input_df.to_csv('toronto_data.csv', index=False, header=None)

### For testing purposes, only. Use to get Toronto location coordinates using geopy. 


In [25]:
# Use this code to get city latitude/longitude data

# NOTE: Nominatim(user_agent="my-application") is recommended on the geopy web site
# import  os

# Use geopy library to get the latitude and longitude values of Toronto, Ontario.
# In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, 
# as shown below.
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, ON are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Toronto, ON are 43.653963, -79.387207.


### Complete portable code to get location data.

In [38]:
#  Once this code is finished, it will output a new CSV file with two columns: latitude and longitude
import geopy
import pandas
from geopy.geocoders import Nominatim
# from geopy.geocoders import GoogleV3
# versions used: geopy 1.10.0, pandas 0.16.2, python 2.7.8

def main():
  # io = pandas.read_csv('census(1)', index_col=None, header=0, sep=",")
  io = pandas.read_csv('toronto_districts_neighborhoods.csv', index_col=None, header=0, sep=",")
    
  def get_latitude(x):
    return x.latitude

  def get_longitude(x):
    return x.longitude

  # uncomment the geolocator you want to use
  geolocator = Nominatim(user_agent="tor_explorer")

  geolocate_column = io['district'].apply(geolocator.geocode)
  io['latitude'] = geolocate_column.apply(get_latitude)
  io['longitude'] = geolocate_column.apply(get_longitude)
  io.to_csv('geocoding-output.csv')

if __name__ == '__main__':
    main()

### Create 'toronto_geoJSON' file using 'geocoding-output.csv' file from above. Then, use www.geojson.io tool to create the geoJSON file.

## Load and explore the toronto_geoJSON data

In [39]:
# import JSON library to handle JSON files
import json

# Next, let's load the data.
with open('toronto_geoJSON.geojson') as json_data:
    toronto_data = json.load(json_data)

# Let's take a quick look at the data.
toronto_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'': 0,
    'name': 'Alexandra Park',
    'district': 'Downtown Core'},
   'geometry': {'type': 'Point', 'coordinates': [-79.40550232, 43.65108871]}},
  {'type': 'Feature',
   'properties': {'': 1,
    'name': 'Baldwin Village',
    'district': 'Downtown Core'},
   'geometry': {'type': 'Point', 'coordinates': [-79.39549, 43.65559]}},
  {'type': 'Feature',
   'properties': {'': 2, 'name': 'Cabbagetown', 'district': 'Downtown Core'},
   'geometry': {'type': 'Point', 'coordinates': [-79.36418915, 43.66759872]}},
  {'type': 'Feature',
   'properties': {'': 3, 'name': 'Chinatown', 'district': 'Downtown Core'},
   'geometry': {'type': 'Point', 'coordinates': [-79.39633942, 43.65330887]}},
  {'type': 'Feature',
   'properties': {'': 4,
    'name': 'Church and Wellesley',
    'district': 'Downtown Core'},
   'geometry': {'type': 'Point', 'coordinates': [-79.38093, 43.6657]}},
  {'type': 'Feature',
   'properties':

In [40]:
# Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods.
# So, let's define a new variable that includes this data.
neighborhoods_data = toronto_data['features']

# Let's take a look at the first item in this list.
neighborhoods_data[0]


{'type': 'Feature',
 'properties': {'': 0, 'name': 'Alexandra Park', 'district': 'Downtown Core'},
 'geometry': {'type': 'Point', 'coordinates': [-79.40550232, 43.65108871]}}

## Tranform the geojson data into a pandas dataframe


In [41]:
# The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe.
# So let's start by creating an empty dataframe.

# define the dataframe columns
column_names = ['District', 'Neighborhood', 'Latitude', 'Longitude'] 


In [42]:
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Take a look at the empty dataframe to confirm that the columns are as intended.
neighborhoods


,District,Neighborhood,Latitude,Longitude


In [43]:
# Then let's loop through the data [from the json file] and fill the dataframe one row at a time.
for data in neighborhoods_data:
    district = neighborhood_name = data['properties']['district'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'District': district,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
# Quickly examine the resulting dataframe.
neighborhoods

,District,Neighborhood,Latitude,Longitude
0,Downtown Core,Alexandra Park,43.651089,-79.405502
1,Downtown Core,Baldwin Village,43.655590,-79.395490
2,Downtown Core,Cabbagetown,43.667599,-79.364189
3,Downtown Core,Chinatown,43.653309,-79.396339
4,Downtown Core,Church and Wellesley,43.665700,-79.380930
5,Downtown Core,Corktown,43.249249,-79.866920
6,Downtown Core,Discovery District,43.692400,-79.577500
7,Downtown Core,Distillery District,43.650558,-79.359718
8,Downtown Core,East Bayfront,43.648689,-79.385437
9,Downtown Core,Financial District,43.648689,-79.385437


In [44]:
# And make sure that the dataset has all 9 districts and 194 neighborhoods.
print('The dataframe has {} districts and {} neighborhoods.'.format(
        len(neighborhoods['District'].unique()),
        neighborhoods.shape[0]
    )
)


The dataframe has 9 districts and 195 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto, Ontario.

In [45]:
# In order to define an instance of the geocoder, we need to define a user_agent. 
# We will name our agent tor_explorer, as shown below.
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [46]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['District'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

In [47]:
# Use Folium. let's simplify the above map and segment and cluster only the neighborhoods in Downtown Core.
# So, let's slice the original dataframe and create a new dataframe of the Downtown Core data.
downtown_core_data = neighborhoods[neighborhoods['District'] == 'Downtown Core'].reset_index(drop=True)

downtown_core_data.head()


,District,Neighborhood,Latitude,Longitude
0,Downtown Core,Alexandra Park,43.651089,-79.405502
1,Downtown Core,Baldwin Village,43.655590,-79.395490
2,Downtown Core,Cabbagetown,43.667599,-79.364189
3,Downtown Core,Chinatown,43.653309,-79.396339
4,Downtown Core,Church and Wellesley,43.665700,-79.380930


In [48]:
# Let's get the geographical coordinates of Alexandra Park.
address = 'Alexandra Park Toronto ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Alexandra Park are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Alexandra Park are 43.65075755, -79.4042978683821.


In [49]:
# As we did with all of Toronto City, let's visualize Downtown Core District and the neighborhoods in it.
# create map of Downtown Core using latitude and longitude values
map_downtown_core = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_core_data['Latitude'], downtown_core_data['Longitude'], downtown_core_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_core)  

map_downtown_core

### Define Foursquare Credentials and Version

In [50]:

# @hidden_cell
# Define Foursquare Credentials and Version
CLIENT_ID = '14KPVCUGAVI4WFWJIIDQUKWTANKMCYVY1LTW0RCZ1LDZPAL3' # your Foursquare ID
CLIENT_SECRET = 'CIPQNCK34AMYB5RFJDTJ0XXJBDZXD2F2EV2KZ1WTNXW2C5M0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 14KPVCUGAVI4WFWJIIDQUKWTANKMCYVY1LTW0RCZ1LDZPAL3
CLIENT_SECRET:CIPQNCK34AMYB5RFJDTJ0XXJBDZXD2F2EV2KZ1WTNXW2C5M0


### Let's explore the first neighborhood in our dataframe.

In [51]:
# Get the neighborhood's name.
downtown_core_data.loc[0, 'Neighborhood']

# Get the neighborhood's latitude and longitude values.
# neighborhood latitude value
neighborhood_latitude = downtown_core_data.loc[0, 'Latitude'] 
# neighborhood longitude value
neighborhood_longitude = downtown_core_data.loc[0, 'Longitude'] 
# neighborhood name
neighborhood_name = downtown_core_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Alexandra Park are 43.65108871, -79.40550232.


In [52]:
# Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
# First, let's create the GET request URL. Name your URL url.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [53]:
# Send the GET request and examine the results
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5cf58901dd57970e25d1e2a2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Alexandra Park',
  'headerFullLocation': 'Alexandra Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 109,
  'suggestedBounds': {'ne': {'lat': 43.655588714500006,
    'lng': -79.39929464794729},
   'sw': {'lat': 43.6465887055, 'lng': -79.4117099920527}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f5e4f67e4b0241544b69415',
       'name': '#Hashtag Gallery',
       'location': {'address': '801 Dundas St W',
        'crossStreet': 'Between Markham & Palmerston',
        'lat': 43.65183024518982,
        'lng': -79.40810273899646,
       

In [54]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
# flatten JSON
nearby_venues = json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,#Hashtag Gallery,Art Gallery,43.651830,-79.408103
1,Montauk,Bar,43.652084,-79.406898
2,Porchetta & Co.,Sandwich Place,43.651576,-79.408950
3,Petit Nuage,Dessert Shop,43.651717,-79.403999
4,Market 707,Food Court,43.652128,-79.404844


In [55]:
# And how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


100 venues were returned by Foursquare.


### 2. Explore Neighborhoods in Downtown Core (Central)

In [56]:
# Let's create a function to repeat the same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


### Now write the code to run the above function on each neighborhood and create a new dataframe called downtown_core_venues.

In [57]:
# the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.
downtown_core_venues = getNearbyVenues(names=downtown_core_data['Neighborhood'],
                                   latitudes=downtown_core_data['Latitude'],
                                   longitudes=downtown_core_data['Longitude']
                                  )


Alexandra Park
Baldwin Village
Cabbagetown
Chinatown
Church and Wellesley
Corktown
Discovery District
Distillery District
East Bayfront
Financial District
Garden District
Grange Park
Harbord Village
Harbourfront
Kensington Market
Little Japan
Moss Park
Old Town
Quayside
Queen Street West
Regent Park
South Core
St. James Town
St. Lawrence
The Annex
The Entertainment District
Toronto Islands
Trefann Court
University
Yorkville
CityPlace
Fashion District


In [58]:
# Let's check the size of the resulting dataframe
print(downtown_core_venues.shape)
downtown_core_venues.head()


(1830, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alexandra Park,43.651089,-79.405502,#Hashtag Gallery,43.651830,-79.408103,Art Gallery
1,Alexandra Park,43.651089,-79.405502,Montauk,43.652084,-79.406898,Bar
2,Alexandra Park,43.651089,-79.405502,Porchetta & Co.,43.651576,-79.408950,Sandwich Place
3,Alexandra Park,43.651089,-79.405502,Petit Nuage,43.651717,-79.403999,Dessert Shop
4,Alexandra Park,43.651089,-79.405502,Market 707,43.652128,-79.404844,Food Court


In [59]:
# Let's check how many venues were returned for each neighborhood
downtown_core_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alexandra Park,100,100,100,100,100,100
Baldwin Village,74,74,74,74,74,74
Cabbagetown,40,40,40,40,40,40
Chinatown,94,94,94,94,94,94
Church and Wellesley,82,82,82,82,82,82
CityPlace,8,8,8,8,8,8
Corktown,27,27,27,27,27,27
Discovery District,19,19,19,19,19,19
Distillery District,46,46,46,46,46,46


### Let's find out how many unique categories can be curated from all the returned venues

In [60]:
print('There are {} uniques categories.'.format(len(downtown_core_venues['Venue Category'].unique())))


There are 222 uniques categories.


### 3. Analyze Each Neighborhood

In [61]:
# one hot encoding
downtown_core_onehot = pd.get_dummies(downtown_core_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_core_onehot['Neighborhood'] = downtown_core_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_core_onehot.columns[-1]] + list(downtown_core_onehot.columns[:-1])
downtown_core_onehot = downtown_core_onehot[fixed_columns]

downtown_core_onehot.head()


,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Tram Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Alexandra Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Alexandra Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [62]:
# And let's examine the new dataframe size.
downtown_core_onehot.shape


(1830, 222)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [63]:
downtown_core_grouped = downtown_core_onehot.groupby('Neighborhood').mean().reset_index()
downtown_core_grouped

# Let's confirm the new size
downtown_core_grouped.shape


(32, 222)

### Let's print each neighborhood along with the top 5 most common venues

In [64]:
num_top_venues = 5

for hood in downtown_core_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_core_grouped[downtown_core_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Alexandra Park----
                           venue  freq
0                            Bar  0.07
1                    Coffee Shop  0.05
2         Furniture / Home Store  0.05
3  Vegetarian / Vegan Restaurant  0.04
4             Italian Restaurant  0.04


----Baldwin Village----
                   venue  freq
0                   Café  0.09
1            Coffee Shop  0.07
2    Dumpling Restaurant  0.05
3         Sandwich Place  0.05
4  Vietnamese Restaurant  0.04


----Cabbagetown----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.05
2         Café  0.05
3         Park  0.05
4  Pizza Place  0.05


----Chinatown----
                 venue  freq
0                 Café  0.07
1          Coffee Shop  0.04
2  Dumpling Restaurant  0.04
3   Chinese Restaurant  0.03
4                  Bar  0.03


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.05
2              Gay Bar  0.04
3                Diner  0.02
4            

### Let's put that into a pandas dataframe

In [65]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_core_grouped['Neighborhood']

for ind in np.arange(downtown_core_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_core_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexandra Park,Bar,Furniture / Home Store,Coffee Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Café,Asian Restaurant,Nightclub,Mediterranean Restaurant
1,Baldwin Village,Café,Coffee Shop,Dumpling Restaurant,Sandwich Place,Bar,Vietnamese Restaurant,Chinese Restaurant,Ramen Restaurant,Arts & Crafts Store,Ice Cream Shop
2,Cabbagetown,Coffee Shop,Café,Park,Pizza Place,Restaurant,Bakery,Building,Bistro,Breakfast Spot,Liquor Store
3,Chinatown,Café,Dumpling Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Chinese Restaurant,Cocktail Bar,Bar,Arts & Crafts Store,Mexican Restaurant,Vietnamese Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Fast Food Restaurant,Sushi Restaurant,Gastropub,Burger Joint,Restaurant,Dance Studio,Diner


### 4. Cluster Neighborhoods

In [66]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

downtown_core_grouped_clustering = downtown_core_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_core_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_core_merged = downtown_core_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_core_merged = downtown_core_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# check the last columns!
downtown_core_merged.head() 


,District,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Core,Alexandra Park,43.651089,-79.405502,0,Bar,Furniture / Home Store,Coffee Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Café,Asian Restaurant,Nightclub,Mediterranean Restaurant
1,Downtown Core,Baldwin Village,43.655590,-79.395490,0,Café,Coffee Shop,Dumpling Restaurant,Sandwich Place,Bar,Vietnamese Restaurant,Chinese Restaurant,Ramen Restaurant,Arts & Crafts Store,Ice Cream Shop
2,Downtown Core,Cabbagetown,43.667599,-79.364189,0,Coffee Shop,Café,Park,Pizza Place,Restaurant,Bakery,Building,Bistro,Breakfast Spot,Liquor Store
3,Downtown Core,Chinatown,43.653309,-79.396339,0,Café,Dumpling Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Chinese Restaurant,Cocktail Bar,Bar,Arts & Crafts Store,Mexican Restaurant,Vietnamese Restaurant
4,Downtown Core,Church and Wellesley,43.665700,-79.380930,0,Coffee Shop,Japanese Restaurant,Gay Bar,Fast Food Restaurant,Sushi Restaurant,Gastropub,Burger Joint,Restaurant,Dance Studio,Diner


In [67]:
# Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_core_merged['Latitude'], downtown_core_merged['Longitude'], downtown_core_merged['Neighborhood'], downtown_core_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# 5. Examine Clusters

In [68]:
# Now, you can examine each cluster and determine the discriminating venue categories that 
# distinguish each cluster. 
# Based on the defining categories, you can then assign a name to each cluster. 

# Cluster 1
downtown_core_merged.loc[downtown_core_merged['Cluster Labels'] == 0, downtown_core_merged.columns[[1] + list(range(5, downtown_core_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexandra Park,Bar,Furniture / Home Store,Coffee Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Café,Asian Restaurant,Nightclub,Mediterranean Restaurant
1,Baldwin Village,Café,Coffee Shop,Dumpling Restaurant,Sandwich Place,Bar,Vietnamese Restaurant,Chinese Restaurant,Ramen Restaurant,Arts & Crafts Store,Ice Cream Shop
2,Cabbagetown,Coffee Shop,Café,Park,Pizza Place,Restaurant,Bakery,Building,Bistro,Breakfast Spot,Liquor Store
3,Chinatown,Café,Dumpling Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Chinese Restaurant,Cocktail Bar,Bar,Arts & Crafts Store,Mexican Restaurant,Vietnamese Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Fast Food Restaurant,Sushi Restaurant,Gastropub,Burger Joint,Restaurant,Dance Studio,Diner
5,Corktown,Pub,Italian Restaurant,Sandwich Place,Park,Café,Bar,Seafood Restaurant,Restaurant,Gastropub,Ethiopian Restaurant
8,East Bayfront,Coffee Shop,Café,Bar,Steakhouse,Hotel,Pizza Place,Pub,Restaurant,Japanese Restaurant,Italian Restaurant
9,Financial District,Coffee Shop,Café,Bar,Steakhouse,Hotel,Pizza Place,Pub,Restaurant,Japanese Restaurant,Italian Restaurant
11,Grange Park,Coffee Shop,Café,Art Gallery,Record Shop,Arts & Crafts Store,Japanese Restaurant,Cocktail Bar,Chinese Restaurant,Sporting Goods Shop,Sushi Restaurant
12,Harbord Village,Café,Coffee Shop,Pizza Place,Bookstore,Sandwich Place,Sushi Restaurant,Restaurant,Bar,Bakery,Beer Bar


### Cluster 2

In [69]:
# Cluster 2
downtown_core_merged.loc[downtown_core_merged['Cluster Labels'] == 1, downtown_core_merged.columns[[1] + list(range(5, downtown_core_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Garden District,Kids Store,Pharmacy,Grocery Store,Park,Bank,Women's Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm


### Cluster 3

In [70]:
# Cluster 3
downtown_core_merged.loc[downtown_core_merged['Cluster Labels'] == 2, downtown_core_merged.columns[[1] + list(range(5, downtown_core_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Toronto Islands,Boat or Ferry,Harbor / Marina,Pier,Café,Women's Store,Electronics Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market


### Cluster 4

In [71]:
# Cluster 4
downtown_core_merged.loc[downtown_core_merged['Cluster Labels'] == 3, downtown_core_merged.columns[[1] + list(range(5, downtown_core_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Discovery District,Coffee Shop,American Restaurant,Sushi Restaurant,Hotel,Restaurant,Rental Car Location,Thai Restaurant,Breakfast Spot,Fast Food Restaurant,Sandwich Place
7,Distillery District,Coffee Shop,Theater,Bakery,Pub,Dessert Shop,Spa,Gastropub,Italian Restaurant,French Restaurant,Boat or Ferry
16,Moss Park,Coffee Shop,Sandwich Place,Italian Restaurant,Dance Studio,Diner,Yoga Studio,Event Space,Café,Rental Car Location,Pub
20,Regent Park,Coffee Shop,Thai Restaurant,Indian Restaurant,Electronics Store,Sushi Restaurant,Food Truck,Beer Store,Restaurant,Auto Dealership,Pool
21,South Core,Thai Restaurant,Restaurant,Coffee Shop,Pharmacy,Grocery Store,Café,Sushi Restaurant,Food Truck,Beer Store,Gastropub
22,St. James Town,Coffee Shop,Pizza Place,Grocery Store,Hostel,Filipino Restaurant,Library,Market,Food & Drink Shop,Breakfast Spot,Pie Shop
23,St. Lawrence,Coffee Shop,Park,Gym / Fitness Center,Pub,Restaurant,Breakfast Spot,Bakery,Health Food Store,Electronics Store,Café
27,Trefann Court,Coffee Shop,Restaurant,Breakfast Spot,Theater,Park,Dance Studio,Pub,Pool,Bakery,Diner
30,CityPlace,Coffee Shop,Greek Restaurant,Chinese Restaurant,Skating Rink,Convenience Store,Sushi Restaurant,Seafood Restaurant,Farm,Falafel Restaurant,Exhibit


### Cluster 5

In [72]:
# Cluster 5
downtown_core_merged.loc[downtown_core_merged['Cluster Labels'] == 4, downtown_core_merged.columns[[1] + list(range(5, downtown_core_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Quayside,Construction & Landscaping,Harbor / Marina,Park,Shopping Mall,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit
